In [1]:
from pykrx import stock
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
import datetime
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kospi_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
kospi_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kosdaq_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="KOSDAQ")})
kosdaq_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))


In [2]:
today = datetime.datetime.today()
month_ago = str(today.year) + str(today.month-1) + str(today.day)
month_ago2 = str(today.year) + str(today.month-2) + str(today.day)
today = str(today.year) + str(today.month) + str(today.day)
month_ago

'20211012'

In [3]:
shrink_kospi = stock.get_market_cap_by_ticker(month_ago, market='KOSPI').sort_values('시가총액', ascending=False)
shrink_kospi = shrink_kospi[0:50]
shrink_kosdaq = stock.get_market_cap_by_ticker(month_ago, market='KOSDAQ').sort_values('시가총액', ascending=False)
shrink_kosdaq = shrink_kosdaq[0:50]
name1 = []
name2 = []
for ticker in shrink_kospi.index:
    name1.append(kospi_list[kospi_list['종목코드'] == ticker]['종목명'].values[0])
for ticker in shrink_kosdaq.index:
    name2.append(kosdaq_list[kosdaq_list['종목코드'] == ticker]['종목명'].values[0])
total_name = name1 + name2
len(name1), len(name2), len(total_name)

(50, 50, 100)

In [4]:
def get_name(name):
    name = name
    fromdate = month_ago2
    todate = today

    ticker = stock_list.loc[stock_list['종목명']==name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker)
    df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
    df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동표준편차
    df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
    df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
    df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드
    df = df[19:] # 20일 이동평균을 구했기 때문에 20번째 행부터 값이 들어가 있음
    df.insert(0, column='volatility', value=0)

    df['volatility'] = (df['upper'] - df['lower'])

    return df[::-1]


In [5]:
select = []
for name in total_name:
    cnt_moment = 0
    mnt = get_name(name)['Close']
    if (mnt[1] > mnt[3]) & (mnt[2] > mnt[4]) & (mnt[3] > mnt[5]):
        select.append(name)

len(select)
        

5

In [6]:
select2 = []
for name in select:
    cnt_moment = 0
    vol = get_name(name)['volatility']
    if (vol[1] > vol[3]) & (vol[2] > vol[4]) & (vol[3] > vol[5]):
        select2.append(name)

select2  

['에디슨EV']

In [7]:
name = '한국전력'
today = datetime.datetime.today()
month_ago = str(today.year) + str(today.month-1) + str(today.day)
month_ago2 = str(today.year) + str(today.month-2) + str(today.day)
today = str(today.year) + str(today.month) + str(today.day)
fromdate = str(20210601)
todate = today

ticker = stock_list.loc[stock_list['종목명']==name, '종목코드']
df = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker)
df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동평균
df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드
df = df[19:] # 20일 이동평균을 구했기 때문에 20번째 행부터 값이 들어가 있음
df['volatility'] = (df['upper'] - df['lower'])/2
df[::-1].head(8)

df1 = df.copy().loc[:,('Close','ma20','volatility')]
df1.insert(0, column='f_close', value=0)
df1.insert(0, column='f_ma20', value=0)
df1.insert(0, column='f_vol', value=0)
df1.insert(0, column='ff_vol', value=0)

for i in range(1, len(df1)):
    df1['f_vol'][i] = df['volatility'][i] - df['volatility'][i-1]
    df1['f_ma20'][i] = df['ma20'][i] - df['ma20'][i-1]
    df1['f_close'][i] = df['Close'][i] - df['Close'][i-1]
for i in range(1, len(df1)):
    df1['ff_vol'][i] = df1['f_vol'][i] - df1['f_vol'][i-1]


scaler = MinMaxScaler()
df2 = df1.copy()
df2[:] = scaler.fit_transform(df2[:])

df2[::-1].head(8)

C:\Users\ghdak\AppData\Local\Temp/ipykernel_19248/2377802709.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['f_vol'][i] = df['volatility'][i] - df['volatility'][i-1]
C:\Users\ghdak\AppData\Local\Temp/ipykernel_19248/2377802709.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['f_ma20'][i] = df['ma20'][i] - df['ma20'][i-1]
C:\Users\ghdak\AppData\Local\Temp/ipykernel_19248/2377802709.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

,ff_vol,f_vol,f_ma20,f_close,Close,ma20,volatility
날짜,,,,,,,
2021-11-11,0.252850,0.705954,0.670659,0.250000,0.161765,0.046117,0.102855
2021-11-10,0.316062,0.719320,0.760479,0.392857,0.279412,0.045307,0.105428
2021-11-09,0.061140,0.658566,1.000000,0.285714,0.338235,0.039644,0.099856
2021-11-08,0.414508,0.896719,0.940120,1.000000,0.441176,0.021036,0.129213
2021-11-05,0.249741,0.720535,0.688623,0.500000,0.250000,0.005663,0.022763
2021-11-04,0.323316,0.737546,0.730539,0.928571,0.264706,0.004045,0.016047
2021-11-03,0.301554,0.668287,0.449102,0.285714,0.102941,0.000000,0.000000
2021-11-02,0.258031,0.624544,0.508982,0.750000,0.205882,0.011327,0.023807


In [8]:
## back testing
